In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential 
from keras.layers import Dense, Activation 
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras import initializers
import math 
from tensorflow import keras
import statistics 
from sklearn import metrics
from sklearn.metrics import classification_report

In [8]:
os.chdir("C:/Users/Yingxing Huang/Desktop/Compsci 361 Machine Learning/A3 Group Project/Data/A3")
"""data preprocessing"""
# load the data
train_data = pd.read_csv ("train.csv")
test_data = pd.read_csv ("test.csv")

"""vectorisation""" 
# combine two train set and test set
full_data = pd.concat([train_data, test_data], axis=0)
text = full_data['Text']
# Term Frequency - Inverse Document Frequency (TF - IDF)
vectorizor = TfidfVectorizer() 
vectorizor.fit (text)
# encode article 
vector = vectorizor.transform(text)

"""recover the split"""
train_X = vector[:428].toarray()
test_X = vector[428:].toarray()
train_y_bool = list(train_data ['Category'] == "tech")
test_y_bool = list(test_data ['Category'] == "tech")
train_y = np.array(list(map(int, train_y_bool)))
test_y =  np.array(list(map(int, test_y_bool)))
train_X.shape


(428, 14927)

# Generate Data

In [9]:
def get_data (p):
    num_obs = round (p * train_X.shape[0])       # the number of instances that will be selected
    new_train_X = train_X [:num_obs]
    new_train_y = train_y[:num_obs]
    new_test_X = train_X [num_obs:]
    new_test_y = train_y[num_obs:]
    return (new_train_X, new_train_y, new_test_X,new_test_y)

# Train the Model

In [56]:
n_features = train_X.shape[1]
train_accuracy = []
test_accuracy = []


m = [0.1, 0.3, 0.5, 0.7, 0.9]
for p in m:
    new_train_X, new_train_y, new_test_X,new_test_y = get_data (p)
    build_NN(20, new_train_X, new_train_y,new_test_X, new_test_y, test_X, test_y)     # use 20 hidden units

print (f'train_accuracy\n {train_accuracy}\n')
print (f'test_accuracy\n {test_accuracy}\n')
   

    


train_accuracy
 [0.9944367861016025, 0.9970300287567164, 0.9980236825517426, 0.9982213163254502, 0.998326005464477]

test_accuracy
 [0.6697214076246335, 0.9615942028985507, 0.9903169818215036, 0.9903732631005359, 0.9903732631005359]



# Plot training accuracy

# Plot testing accuracy

# Build Neural Network 

In [55]:
def build_NN (h,new_train_X, new_train_y,new_test_X, new_test_y, test_X, test_y):
    initializer = initializers.RandomUniform(minval = 0, maxval = 0.1, seed=None)
    nn_model = Sequential ()
    nn_model.add (Dense(h, input_dim = n_features, activation = 'relu', kernel_initializer = initializer) )  # hidden layer: a dense layer that is fully connected
    nn_model.add(Dense(1, activation = 'sigmoid', kernel_initializer = initializer))   # ouput layer
    
    # define optimiser and initialise weights 
    opt = keras.optimizers.Adam(learning_rate = 0.01)
    # compile the model
    nn_model.compile (loss = 'binary_crossentropy', optimizer = opt, metrics = ['Precision', 'Recall'])

    # fit the model
    history = nn_model.fit (new_train_X, new_train_y, epochs = 100 , batch_size = 10, verbose = 0)
    
    # train accuracy 
    f1 = calculate_f1 (statistics.mean(history.history['precision']), statistics.mean(history.history['recall']))
    train_accuracy.append(f1)   # f1 is a list
    
    # validation accuracy
    #_, accuracy = nn_model.evaluate(new_test_X, new_test_y)
    

    # test accuracy
    y_pred = nn_model.predict_classes(test_X)
    classi_report = classification_report(test_y, y_pred, output_dict = True) # find f1 score
    test_accuracy.append(classi_report['macro avg']['f1-score'])
    

In [52]:
def calculate_f1 (precision, recall):
    f1  = 2 * precision * recall / (precision + recall)
    
    return f1